In [3]:
import numpy as np

from elarb.optimiser import optimise, Instance
from elarb.models import SolarPanel, Battery, GridConnection, Inverter, Facility

# solar panels: 470Wp JinKO TigerNeo N-Type 60HL4 BF, 1.57 kWh / time, costs 1565 DKK
panel = SolarPanel(m2=0.75, depreciation_per_hour=0.006)
# Deye SUN-12K-SG04LP3-EU
inverter = Inverter(depreciation_per_hour=0.05, throughput_kWh=15.6, conversion_loss_pct=0.03)
# battery: Powerwall LBATTS Powerwall, 8.8 kwh
# bat_cost_dkk = 21000; bat_cycles = 6000; bat_depreciation = np.round(bat_cost_dkk / (8.8*bat_cycles), 3)
battery = Battery(depreciation_per_kWh=0.3, throughput_kWh=3.3, capacity_kWh=8.8, conversion_loss_pct=0.03)
# DK grid, udvidet stikledning
amp = 63
volt = 400
grid = GridConnection(throughput_kWh=amp * volt / 1000)

# Connect components in a facility
facility=Facility(
    panel=panel,
    battery=battery,
    inverter=inverter,
    grid=grid,
    n_panels=12,
    n_batteries=1,
    n_inverters=1,
)


spot_price = np.array([
    2.94, 2.73, 2.72, 3.01, 3.03, 3.44, 3.65, 3.82, 3.75, 3.61, 3.43,
    3.16, 3.16, 1.47, 1.47, 1.47, 1.47, 1.47, 3.03, 3.15, 3.07, 2.89,
    2.5 , 2.3 , 2.  , 1.76, 1.84, 2.17, 2.36, 2.89, 3.17, 3.38, 3.48,
    3.47, 3.47, 3.29])

pv_kwh = np.array([
    0.32, 0.27, 0.2 , 0.13, 0.09, 0.06, 0.02, 0.  , 0.  , 0.  , 0.  ,
    0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.02, 0.07, 0.16, 0.3 , 0.47,
    0.48, 0.46, 0.33, 0.34, 0.28, 0.23, 0.16, 0.1 , 0.04, 0.01, 0.  ,
    0.  , 0.  , 0.  ])
# https://radiuselnet.dk/elnetkunder/tariffer-og-netabonnement/
net_tariff = np.array([
    0.25, 0.25, 0.25, 0.25, 0.25, 0.66, 0.66, 0.66, 
    0.66, 0.25, 0.25, 0.25, 0.17, 0.17, 0.17, 0.17, 
    0.17, 0.17, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 
    0.25, 0.25, 0.25, 0.25, 0.25, 0.66, 0.66, 0.66, 
    0.66, 0.25, 0.25, 0.25, ])
spot_demand_kWh = np.zeros(36) + 9999
spot_supply_kWh = np.zeros(36) + 9999

instance = Instance(
    facility=facility,
    spot_price=spot_price,
    pv_dc_kWh_m2=pv_kwh,
    net_tariff=net_tariff,
    spot_demand_kWh=spot_demand_kWh,
    spot_supply_kWh=spot_supply_kWh,
    initial_soc=0.0
)

res = optimise(instance)

print()
print('Result')
print('- Profit:', np.round(res.value, 3))
print('- PV contribution:', np.round(res.x1_contrib, 3))
print('- X2 contribution:', np.round(res.x2_contrib, 3))
print('- X3 contribution:', np.round(res.x3_contrib, 3))
print('- X4 contribution:', np.round(res.x4_contrib, 3))
print('- Panel deprecation:', np.round(res.panel_depreciation, 3))
print('- Inverter deprecation:', np.round(res.inverter_depreciation, 3))
print('- Battery deprecation:', np.round(res.battery_depreciation, 3))
print('- Battery soc:', res.battery_soc_kWh.round(3))


Result
- Profit: 108.738
- PV contribution: 56.369
- X2 contribution: 0.0
- X3 contribution: 79.023
- X4 contribution: -14.574
- Panel deprecation: 2.592
- Inverter deprecation: 1.8
- Battery deprecation: 7.688
- Battery soc: [0.    2.794 5.151 6.897 8.032 8.032 8.032 8.032 6.6   6.6   3.3   0.
 0.    0.    2.081 4.035 6.198 8.62  8.795 6.6   3.3   0.    0.    0.
 0.    2.881 5.849 8.294 8.785 8.8   8.8   8.8   8.8   8.8   5.5   2.2  ]
